In [1]:
from flask import Flask, render_template, jsonify
import pandas as pd
from sqlalchemy import create_engine, func
import os
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from datetime import datetime as dt

In [2]:
engine_sqlite = create_engine('sqlite:///teen_births.sqlite')

In [10]:
session = Session(engine_sqlite)

In [29]:
df=pd.read_sql("select * from birth_rates", engine_sqlite)
df

,year,state,age_group,state_rate,state_births,us_births,us_birth_rate,unit
0,1990,Alabama,15-17 years,47.4,"4,222","183,327",37.5,"per 1,000"
1,1990,Alaska,15-17 years,31.2,335,"183,327",37.5,"per 1,000"
2,1990,Arizona,15-17 years,47.7,"3,436","183,327",37.5,"per 1,000"
3,1990,Arkansas,15-17 years,50.4,"2,549","183,327",37.5,"per 1,000"
4,1990,California,15-17 years,44.6,"24,880","183,327",37.5,"per 1,000"
...,...,...,...,...,...,...,...,...
4675,2019,Virginia,18-19 years,25.1,"2,800","130,593",31.1,"per 1,000"
4676,2019,Washington,18-19 years,25.4,"2,189","130,593",31.1,"per 1,000"
4677,2019,West Virginia,18-19 years,49.3,"1,009","130,593",31.1,"per 1,000"
4678,2019,Wisconsin,18-19 years,24.0,"1,835","130,593",31.1,"per 1,000"


In [37]:
df[(df.year == 1990) & (df.state == "Alaska")]

,year,state,age_group,state_rate,state_births,us_births,us_birth_rate,unit
1,1990,Alaska,15-17 years,31.2,335,"183,327",37.5,"per 1,000"
1353,1990,Alaska,15-19 years,65.3,"1,142","521,826",59.9,"per 1,000"
2705,1990,Alaska,18-19 years,120.0,807,"338,499",88.6,"per 1,000"


In [49]:
df.groupby(["state"]).mean()

,year,state_rate,us_birth_rate
state,,,
Alabama,2004.5,54.233333,43.438889
Alaska,2004.5,48.153333,43.438889
Arizona,2004.5,58.157778,43.438889
Arkansas,2004.5,62.330000,43.438889
California,2004.5,45.073333,43.438889
Colorado,2004.5,42.263333,43.438889
Connecticut,2004.5,26.430000,43.438889
Delaware,2004.5,40.873333,43.438889
District of Columbia,2004.5,54.276667,43.438889


In [51]:
df.dtypes

year               int64
state             object
age_group         object
state_rate       float64
state_births      object
us_births         object
us_birth_rate    float64
unit              object
dtype: object

In [4]:
base = automap_base()
base

sqlalchemy.ext.automap.Base

In [5]:
base.prepare(engine_sqlite, reflect=True)

In [6]:
mapped_classes = base.metadata.tables.keys()
mapped_classes

dict_keys(['birth_rates'])

In [8]:
birth_rates = base.metadata.tables['birth_rates']
birth_rates

Table('birth_rates', MetaData(), Column('year', BIGINT(), table=<birth_rates>), Column('state', TEXT(), table=<birth_rates>), Column('age_group', TEXT(), table=<birth_rates>), Column('state_rate', FLOAT(), table=<birth_rates>), Column('state_births', TEXT(), table=<birth_rates>), Column('us_births', TEXT(), table=<birth_rates>), Column('us_birth_rate', FLOAT(), table=<birth_rates>), Column('unit', TEXT(), table=<birth_rates>), schema=None)

In [15]:
results = session.query(birth_rates.columns.state_rate, birth_rates.columns.state).\
    filter(birth_rates.columns.year == "1990").all()
results

[(47.4, 'Alabama'),
 (31.2, 'Alaska'),
 (47.7, 'Arizona'),
 (50.4, 'Arkansas'),
 (44.6, 'California'),
 (33.1, 'Colorado'),
 (26.4, 'Connecticut'),
 (38.4, 'Delaware'),
 (88.4, 'District of Columbia'),
 (44.9, 'Florida'),
 (50.1, 'Georgia'),
 (32.5, 'Hawaii'),
 (26.3, 'Idaho'),
 (40.1, 'Illinois'),
 (35.3, 'Indiana'),
 (20.4, 'Iowa'),
 (30.4, 'Kansas'),
 (40.8, 'Kentucky'),
 (49.5, 'Louisiana'),
 (23.3, 'Maine'),
 (33.5, 'Maryland'),
 (23.7, 'Massachusetts'),
 (36.0, 'Michigan'),
 (19.9, 'Minnesota'),
 (57.5, 'Mississippi'),
 (39.3, 'Missouri'),
 (24.0, 'Montana'),
 (23.0, 'Nebraska'),
 (42.5, 'Nevada'),
 (17.1, 'New Hampshire'),
 (24.4, 'New Jersey'),
 (46.9, 'New Mexico'),
 (27.5, 'New York'),
 (44.9, 'North Carolina'),
 (15.6, 'North Dakota'),
 (34.3, 'Ohio'),
 (38.8, 'Oklahoma'),
 (30.7, 'Oregon'),
 (28.4, 'Pennsylvania'),
 (31.6, 'Rhode Island'),
 (47.0, 'South Carolina'),
 (23.9, 'South Dakota'),
 (45.0, 'Tennessee'),
 (48.0, 'Texas'),
 (37.5, 'Total U.S.'),
 (26.3, 'Utah'),
 (19

In [13]:
help(session.query(birth_rates.columns.state_rate).filter())

Help on Query in module sqlalchemy.orm.query object:

class Query(sqlalchemy.sql.selectable._SelectFromElements, sqlalchemy.sql.annotation.SupportsCloneAnnotations, sqlalchemy.sql.selectable.HasPrefixes, sqlalchemy.sql.selectable.HasSuffixes, sqlalchemy.sql.selectable.HasHints, sqlalchemy.sql.base.Executable)
 |  Query(entities, session=None)
 |  
 |  ORM-level SQL construction object.
 |  
 |  :class:`_query.Query`
 |  is the source of all SELECT statements generated by the
 |  ORM, both those formulated by end-user query operations as well as by
 |  high level internal operations such as related collection loading.  It
 |  features a generative interface whereby successive calls return a new
 |  :class:`_query.Query` object, a copy of the former with additional
 |  criteria and options associated with it.
 |  
 |  :class:`_query.Query` objects are normally initially generated using the
 |  :meth:`~.Session.query` method of :class:`.Session`, and in
 |  less common cases by instantiat

In [25]:
results = session.query(birth_rates.columns.state_rate, birth_rates.columns.state, birth_rates.columns.year).all()
results

[(47.4, 'Alabama', 1990),
 (31.2, 'Alaska', 1990),
 (47.7, 'Arizona', 1990),
 (50.4, 'Arkansas', 1990),
 (44.6, 'California', 1990),
 (33.1, 'Colorado', 1990),
 (26.4, 'Connecticut', 1990),
 (38.4, 'Delaware', 1990),
 (88.4, 'District of Columbia', 1990),
 (44.9, 'Florida', 1990),
 (50.1, 'Georgia', 1990),
 (32.5, 'Hawaii', 1990),
 (26.3, 'Idaho', 1990),
 (40.1, 'Illinois', 1990),
 (35.3, 'Indiana', 1990),
 (20.4, 'Iowa', 1990),
 (30.4, 'Kansas', 1990),
 (40.8, 'Kentucky', 1990),
 (49.5, 'Louisiana', 1990),
 (23.3, 'Maine', 1990),
 (33.5, 'Maryland', 1990),
 (23.7, 'Massachusetts', 1990),
 (36.0, 'Michigan', 1990),
 (19.9, 'Minnesota', 1990),
 (57.5, 'Mississippi', 1990),
 (39.3, 'Missouri', 1990),
 (24.0, 'Montana', 1990),
 (23.0, 'Nebraska', 1990),
 (42.5, 'Nevada', 1990),
 (17.1, 'New Hampshire', 1990),
 (24.4, 'New Jersey', 1990),
 (46.9, 'New Mexico', 1990),
 (27.5, 'New York', 1990),
 (44.9, 'North Carolina', 1990),
 (15.6, 'North Dakota', 1990),
 (34.3, 'Ohio', 1990),
 (38.8, 'O

In [33]:
start_end_births = session.query(func.avg(birth_rates.columns.state_births),birth_rates.columns.state, birth_rates.columns.year).\
    group_by(birth_rates.columns.year, birth_rates.columns.state).all()
start_end_births

[(7.333333333333333, 'Alabama', 1990),
 (381.0, 'Alaska', 1990),
 (6.0, 'Arizona', 1990),
 (4.333333333333333, 'Arkansas', 1990),
 (45.666666666666664, 'California', 1990),
 (3.3333333333333335, 'Colorado', 1990),
 (2.3333333333333335, 'Connecticut', 1990),
 (426.0, 'Delaware', 1990),
 (281.3333333333333, 'District of Columbia', 1990),
 (17.666666666666668, 'Florida', 1990),
 (11.666666666666666, 'Georgia', 1990),
 (221.33333333333334, 'Hawaii', 1990),
 (204.66666666666666, 'Idaho', 1990),
 (16.0, 'Illinois', 1990),
 (8.0, 'Indiana', 1990),
 (2.0, 'Iowa', 1990),
 (2.6666666666666665, 'Kansas', 1990),
 (6.0, 'Kentucky', 1990),
 (7.666666666666667, 'Louisiana', 1990),
 (190.66666666666666, 'Maine', 1990),
 (5.0, 'Maryland', 1990),
 (4.333333333333333, 'Massachusetts', 1990),
 (13.333333333333334, 'Michigan', 1990),
 (3.0, 'Minnesota', 1990),
 (5.333333333333333, 'Mississippi', 1990),
 (7.0, 'Missouri', 1990),
 (444.0, 'Montana', 1990),
 (243.66666666666666, 'Nebraska', 1990),
 (297.33333

In [57]:
start_end_births = session.query(func.avg(birth_rates.columns.state_births),birth_rates.columns.state,birth_rates.columns.year).\
    filter(birth_rates.columns.state== "Alabama").filter(birth_rates.columns.year=="1990").all()
start_end_births

[(7.333333333333333, 'Alabama', 1990)]

In [58]:
type(start_end_births)

list

In [60]:
year_start = session.query(func.min(birth_rates.columns.state_births),func.max(birth_rates.columns.state_births),birth_rates.columns.state, birth_rates.columns.year).\
    group_by(birth_rates.columns.year, birth_rates.columns.state).all()
year_start

[('11,252', '7,030', 'Alabama', 1990),
 ('1,142', '807', 'Alaska', 1990),
 ('3,436', '9,612', 'Arizona', 1990),
 ('2,549', '7,011', 'Arkansas', 1990),
 ('24,880', '69,712', 'California', 1990),
 ('2,061', '5,975', 'Colorado', 1990),
 ('1,519', '4,038', 'Connecticut', 1990),
 ('1,277', '815', 'Delaware', 1990),
 ('1,189', '841', 'District of Columbia', 1990),
 ('17,099', '9,918', 'Florida', 1990),
 ('11,491', '6,878', 'Georgia', 1990),
 ('1,461', '661', 'Hawaii', 1990),
 ('1,398', '611', 'Idaho', 1990),
 ('15,900', '9,067', 'Illinois', 1990),
 ('12,335', '8,205', 'Indiana', 1990),
 ('1,120', '3,989', 'Iowa', 1990),
 ('1,449', '4,722', 'Kansas', 1990),
 ('3,204', '9,349', 'Kentucky', 1990),
 ('12,270', '7,617', 'Louisiana', 1990),
 ('1,287', '570', 'Maine', 1990),
 ('2,866', '8,143', 'Maryland', 1990),
 ('2,502', '7,266', 'Massachusetts', 1990),
 ('13,305', '7,007', 'Michigan', 1990),
 ('1,650', '5,342', 'Minnesota', 1990),
 ('3,544', '8,909', 'Mississippi', 1990),
 ('11,227', '7,286', '